In [ ]:
!pip install geomltoolkits

In [1]:
import os
from geomltoolkits.downloader import tms as TMSDownloader
from geomltoolkits.downloader import osm as OSMDownloader
from geomltoolkits.regularizer import VectorizeMasks

/Users/krschap/hotosm/geoml-toolkits/src/geomltoolkits/regularizer/orthogonalize.py:36: FutureWarning: This function has no longer any effect, and will be removed in a future release. Starting with Shapely 2.0, equivalent speedups are always available
  speedups.enable()


## Define area of interest to work on 

- Go to Open Aerial Map and then find TMS (Copy TMS Link)
    For this example we are using this image : https://map.openaerialmap.org/#/85.52534580230713,27.63552012302292,14/square/12313122310120202/62d86c65d8499800053796c4?_k=okhkbl 

- Verify OSM is mapped properly in that area
    Go to www.osm.org  , You can simply switch layer in OAM itself and see osm underneath 
    
- Define area of interest where you want to generate training data 
    You can get your bbox from your geojson / area from here : https://kshitijrajsharma.github.io/geojson2bbox/ 

In [ ]:
# Default parameters
ZOOM = 19
WORK_DIR = "banepa"
TMS = "https://tiles.openaerialmap.org/62d85d11d8499800053796c1/0/62d85d11d8499800053796c2/{z}/{x}/{y}"
# AOI = "input.geojson"
bbox = [85.51678033745037,27.6313353660439,85.52323021107895,27.637438390948745]

# Ensure working directory
os.makedirs(WORK_DIR, exist_ok=True)

### Generate chips images for your area of interest

In [ ]:
# Download tiles
await TMSDownloader.download_tiles(
        tms=TMS, 
        zoom=ZOOM, 
        out=WORK_DIR, 
        bbox=bbox, 
        georeference=True, 
        dump_tile_geometries_as_geojson=True,
        prefix="OAM"
    )


### Fetch labels 

Here we are using the same tiles.geojson that was used to fetch tiles , remember your bbox / geojson and tile boundary might be little bit different due to tile size and we want to make sure we have labels for the tile boundary hence input here would be tiles.geojson 

In [ ]:
# Download OSM data for tile boundary
tiles_geojson = os.path.join(WORK_DIR, "tiles.geojson")
await OSMDownloader.download_osm_data(
    geojson=tiles_geojson, 
    out=os.path.join(WORK_DIR, "labels"),
    dump_results=True,
    split_output_by_tiles=True,
    split_prefix="OAM"
)


OSM Data Last Updated :  2025-01-23 12:58:06+00:00
Dumping GeoJSON data to file... banepa/labels/osm-result.geojson
Spllited GeoJSON wrt tiles saved to:  banepa/labels/split


'banepa/labels'

### Regularize Footprints

To regularize building footprints, you can use the `VectorizeMasks` class from the `regularizer` module. Here is an example:

In [ ]:
import os
from geomltoolkits.regularizer import VectorizeMasks

# Define input and output files
input_tiff = "path/to/input.tiff"
output_geojson = "path/to/output.geojson"

# Create a VectorizeMasks instance
converter = VectorizeMasks(
    simplify_tolerance=0.2,
    min_area=1.0,
    orthogonalize=True,
    algorithm="potrace",
    tmp_dir=os.getcwd()
)

# Run the conversion
converter.convert(input_tiff, output_geojson)
